<a href="https://colab.research.google.com/github/IeroK/FineTune/blob/main/Lorabo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
# Disable wandb
import os
os.environ["WANDB_DISABLED"] = "true"

# Uninstall existing packages
!pip uninstall accelerate peft bitsandbytes transformers trl -y

# Install specific versions of packages
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0
!pip install huggingface_hub

Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.44.1
Uninstalling bitsandbytes-0.44.1:
  Successfully uninstalled bitsandbytes-0.44.1
Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Successfully uninstalled transformers-4.46.2
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached tr

In [3]:
# Import necessary libraries
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [4]:
# Load the pre-trained model with 4-bit quantization
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=getattr(torch, "float16"),
        bnb_4bit_quant_type="nf4"
    )
)
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

# Load the tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
    trust_remote_code=True
)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

#Define training arguments with reduced batch size and mixed precision
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,            # Reduced batch size
    gradient_accumulation_steps=1,            # Gradient accumulation
    max_steps=100,
    fp16=True,                                # Enable mixed precision
    logging_strategy="no",                    # Disable logging
    report_to=[],                             # Disable reporting to any tracking tools
    #max_seq_length=512,                     # If applicable, set max_seq_length
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Initialize the SFTTrainer
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=load_dataset(
        path="aboonaji/wiki_medical_terms_llam2_format",
        split="train"
    ),
    tokenizer=llama_tokenizer,
    peft_config=LoraConfig(
        task_type="CAUSAL_LM",
        r=32,
        lora_alpha=0,
        lora_dropout=0.05
    ),
    dataset_text_field="text"
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [6]:
# Start training
llama_sft_trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=2.274801788330078, metrics={'train_runtime': 196.4415, 'train_samples_per_second': 0.509, 'train_steps_per_second': 0.509, 'total_flos': 2871635870121984.0, 'train_loss': 2.274801788330078, 'epoch': 0.014575134819997084})

In [8]:
# Generate text using the fine-tuned model
user_prompt = "Please tell me about Bursitis"
text_generation_pipeline = pipeline(
    task="text-generation",
    model=llama_model,
    tokenizer=llama_tokenizer,
    max_length=600
)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] Please tell me about Bursitis [/INST]  Bursitis is a condition that occurs when the bursae, which are fluid-filled sacs that cushion and reduce friction between tendons, ligaments, and bones, become inflamed or irritated. everybody experiences bursitis at some point in their lives, but it is more common in certain age groups or populations. Here are some common causes of bursitis:

1. Overuse or repetitive motion: Bursitis can occur when a particular joint or area of the body is subjected to repetitive motion or overuse, leading to inflammation and irritation of the bursae. This is common in athletes or individuals who engage in activities that involve repetitive motions, such as running, cycling, or rowing.
2. Trauma or injury: Direct blows to the affected area, such as a fall or a sudden twisting motion, can cause bursitis.
3. Age: As we age, the bursae can become less flexible and more prone to inflammation, making older adults more susceptible to bursitis.
4. Medical cond